In [1]:
PYTHON_PATHS = [".."]
import sys
for path in PYTHON_PATHS:
    if path not in sys.path:
        sys.path.append(path)
import pytorch_lightning as pl
import os
from pytorch_lightning.utilities.parsing import AttributeDict
from pytorch_lightning.loggers import TensorBoardLogger

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
from lieposenet.utils.universal_factory import UniversalFactory
from lieposenet.models.pose_net import PoseNet
from lieposenet.criterions.pose_net_criterion import PoseNetCriterion
from lieposenet.criterions.se3_criterion import SE3Criterion
from lieposenet.data import SevenScenesDataModule

factory = UniversalFactory([PoseNet, PoseNetCriterion, SE3Criterion])

In [4]:
from clearml import Task

/usr/local/lib/python3.9/dist-packages/clearml/backend_interface/task/models.py:38: DeprecationWarning: Flags not at the start of the expression '((?i)(Using model id' (truncated)
  _input_models_re = re.compile(r"((?i)(Using model id: )(\w+)?)")


In [5]:
task = Task.init(project_name="lie-pose-net", task_name="lie-pose-net on local machine")

ClearML Task: created new task id=9207db82732c40e89f30969d61ec3a64


/usr/local/lib/python3.9/dist-packages/nbconvert/exporters/html.py:110: DeprecationWarning:

'contextfilter' is renamed to 'pass_context', the old name will be removed in Jinja 3.1.



======> WARNING! Git diff to large to store (5319kb), skipping uncommitted changes <======
ClearML new version available: upgrade to v1.0.5 is recommended!
ClearML results page: https://app.community.clear.ml/projects/04b664f021694b21aadb7a6f50ab46f1/experiments/9207db82732c40e89f30969d61ec3a64/output/log


/usr/local/lib/python3.9/dist-packages/IPython/core/inputsplitter.py:21: DeprecationWarning:

IPython.core.inputsplitter is deprecated since IPython 7 in favor of `IPython.core.inputtransformer2`



# Load datamodule

In [6]:
data_module_params = {
    "batch_size": 64,
    "use_test": True,
    "num_workers": 4,
    "image_size": 256,
    "scene": "fire",
    "data_path": "/media/mikhail/Data3T/7scenes"
}
task.connect(data_module_params)
scene = data_module_params["scene"]
data_module = SevenScenesDataModule(**data_module_params)

[ToyDataModule] - train subset size 2000
[ToyDataModule] - validation dataset size 2000


# Model

In [ ]:
model_params =  AttributeDict(**{
    "name": "PoseNet",
    "feature_extractor": AttributeDict(
        pretrained=True
    ),
    "criterion": {
        "name": "SE3Criterion",
        "rotation_koef": -3.0,
        "translation_koef": -3.0,
    },
    "feature_dimension": 2048,
    "drop_rate": 0,
    "optimizer": AttributeDict(
        betas="0.9 0.999",
        lr=0.0001,
        weight_decay=0.0005,
    ),
    "scheduler": {
        "step_size": 20,
        "gamma": 0.5,
    },
    "bias": True,
    "activation": "tanh",
    "pretrained": True
})
task.connect(model_params)
model = factory.make_from_parameters(model_params)

/usr/local/lib/python3.9/dist-packages/nbconvert/exporters/html.py:135: DeprecationWarning:

'jinja2.Markup' is deprecated and will be removed in Jinja 3.1. Import 'markupsafe.Markup' instead.

/usr/local/lib/python3.9/dist-packages/nbconvert/exporters/html.py:135: DeprecationWarning:

'jinja2.Markup' is deprecated and will be removed in Jinja 3.1. Import 'markupsafe.Markup' instead.

/usr/local/lib/python3.9/dist-packages/nbconvert/filters/ansi.py:60: DeprecationWarning:

'jinja2.escape' is deprecated and will be removed in Jinja 3.1. Import 'markupsafe.escape' instead.

/usr/local/lib/python3.9/dist-packages/nbconvert/filters/ansi.py:60: DeprecationWarning:

'jinja2.escape' is deprecated and will be removed in Jinja 3.1. Import 'markupsafe.escape' instead.

/usr/local/lib/python3.9/dist-packages/nbconvert/filters/ansi.py:60: DeprecationWarning:

'jinja2.escape' is deprecated and will be removed in Jinja 3.1. Import 'markupsafe.escape' instead.

/usr/local/lib/python3.9/dist-packages/

2021-09-01 15:44:36,573 - clearml.model - INFO - Selected model id: a22c91c76a0f4ef791b13c6d45497aa2


# Train

In [ ]:
logger_path = os.path.join(os.path.dirname(task.cache_dir), "lightning_logs", "lieposenet")
# logger_path = 'lightning_logs'
trainer_params = {
    "max_epochs": 10,
    "checkpoint_every_n_val_epochs": 10,
    "gpus": 1,
    "check_val_every_n_epoch": 2
}
task.connect(trainer_params)
model_checkpoint = pl.callbacks.ModelCheckpoint(monitor='val_loss',
    every_n_val_epochs=trainer_params["checkpoint_every_n_val_epochs"])
trainer = factory.kwargs_function(pl.Trainer)(
    logger=TensorBoardLogger(logger_path, name=SCENE),
    callbacks=[model_checkpoint],
    **trainer_params
)

In [ ]:
trainer.fit(model, data_module)

In [ ]:
trainer.test(model, data_module.test_dataloader())

In [ ]:
task.close()